In [85]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse

In [86]:
df_train = pd.read_csv(r'..\...\treino\r2_gl_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Quero saber é da lei pros mandantes dos jogos ...,r2_gl,0,no,r2_gl_1077,_1273217811075211266,4565,"[('quero', 'V'), ('saber', 'V'), ('é', 'V'), (...",[],"{'mandantes': 1, 'clubes': 1, 'lei': 1, 'monop..."
1,Como não amar a carinha de deboche do golpista...,r2_gl,0,no,r2_gl_652,_1189687806811095041,3759,"[('como', 'KS'), ('não', 'ADV'), ('amar', 'V')...",[],"{'deboche': 1, 'bolsonaro': 1, 'adooooro': 1, ..."
2,"Aquela emissora empregada da Globo, que se cha...",r2_gl,1,no,r2_gl_78,_1254973802213183489,3725,"[('aquela', 'PROADJ'), ('emissora', 'N'), ('em...",[],"{'fazenda': 1, 'emissora': 1, 'edição': 1, 'gl..."
3,RRRRREDE GLOBO APRESENTA,r2_gl,1,no,r2_gl_1258,_1257399816545591297,1951,"[('rrrrrede', 'V'), ('globo', 'N'), ('apresent...",[],{'globo': 1}
4,Gente a própria Globo já se pronunciou negando...,r2_gl,1,no,r2_gl_1338,_1270838107739389952,261,"[('gente', 'N'), ('a', 'ART'), ('própria', 'PR...",[],"{'ataque': 1, 'motivos': 1, 'gente': 1, 'motiv..."
...,...,...,...,...,...,...,...,...,...,...
4504,a globo nao escolheu a sophia pra ganhar a dan...,r2_gl,1,no,r2_gl_1185,_861335540485980163,1845,"[('a', 'PREP'), ('globo', 'N'), ('nao', 'ADV')...",[],"{'sophia': 1, 'povo': 1, 'dança': 1, 'globo': 1}"
4505,Pergunta ao MP RJ onde está a investigação que...,r2_gl,0,no,r2_gl_430,_1086975402407546885,151,"[('pergunta', 'V'), ('ao', 'PREP+ART'), ('mp',...",[],"{'receita': 1, 'mp': 1, '2017': 1, 'rj': 1, 'i..."
4506,Tiradentes morre pela luta por liberdade... Co...,r2_gl,1,no,r2_gl_648,_719719176810668032,1919,"[('tiradentes', 'N'), ('morre', 'V'), ('pela',...",[],"{'liberdade': 1, 'tiradentes': 1, 'trama': 1, ..."
4507,"é algo excepcional, de verdade, enredo de nove...",r2_gl,1,no,r2_gl_1244,_1255667920413351936,4143,"[('é', 'V'), ('algo', 'PROSUB'), ('excepcional...",[],"{'sucesso': 1, 'minissérie': 1, 'vontade': 1, ..."


In [87]:
df_test = pd.read_csv(r'..\...\Teste\r2_gl_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,"Olha só, extrema-direita, se vcs quiserem entr...",r2_gl,0,no,r2_gl_266,_1057399541379616768,760,"[('olha', 'V'), ('só', 'PDEN'), (',', 'PU'), (...",[],"{'mentes': 1, 'lei': 1, 'falha': 1, 'guerra': ..."
1,UFC ta começando na Globoo kkkkkkkkkkkkkkk Ago...,r2_gl,1,no,r2_gl_791,_297945541030072320,1768,"[('ufc', 'N'), ('ta', 'V'), ('começando', 'V')...",[],"{'ufc': 1, 'kkkkkkkkkkkkkkk': 1, 'globoo': 1}"
2,Faz anos que eu boicoto a globo,r2_gl,0,no,r2_gl_1138,_1063859144027979776,815,"[('faz', 'V'), ('anos', 'N'), ('que', 'KS'), (...",[],"{'anos': 1, 'globo': 1}"
3,Foi denunciado de assédio e violência por 12 m...,r2_gl,0,yes,r2_gl_93,_1320388211089309697,1527,"[('@', 'NUM'), ('leviriet_', 'N'), ('foi', 'V'...",['leviriet_'],"{'violência': 1, 'leviriet_': 1, 'assédio': 1,..."
4,Vou entrar nessa também rs obrigado rede globo...,r2_gl,1,no,r2_gl_276,_972309605484236800,408,"[('vou', 'V'), ('entrar', 'V'), ('nessa', 'PRE...",[],"{'rede': 1, 'rs': 1, 'globo': 1}"
...,...,...,...,...,...,...,...,...,...,...
1499,Eu to rindo d+ dos gado achando q o “império” ...,r2_gl,1,no,r2_gl_1130,_1189497137119318016,3239,"[('eu', 'PROPESS'), ('to', 'PROPESS'), ('rindo...",[],"{'mt': 2, 'ces': 1, 'q': 1, 'né': 1, 'aheusnau..."
1500,"Eu ia ate dormir, mas esse filme que começou n...",r2_gl,1,no,r2_gl_228,_540729198592413696,537,"[('eu', 'PROPESS'), ('ia', 'V'), ('ate', 'ADV'...",[],"{'globo': 1, 'filme': 1}"
1501,To rindo com esse filme da globo kkkkkkkkkk,r2_gl,1,no,r2_gl_895,_1090786499325497345,1463,"[('to', 'PROPESS'), ('rindo', 'V'), ('com', 'P...",[],{'filme': 1}
1502,"ESSE FINAL FOI FODA, mas os crente vao atacar ...",r2_gl,1,no,r2_gl_640,_1198065799027732480,33,"[('esse', 'PROADJ'), ('final', 'N'), ('foi', '...",[],"{'kkkkkkkkk': 1, 'crente': 1, 'globo': 1, 'fin..."


In [88]:
#carregando unlabeled data
df_u = pd.read_csv(r'..\...\Unlabeled\globo.csv')
df_u

,Unnamed: 0,Text
0,0,Bobeirada esses filmes da globo
1,1,"a cravícula dela é feia,o globo ocular,a larin..."
2,2,Eu ganhei o celular já com uma conta Globo pla...
3,3,"A Globo tá sem programação, acho que por isso ..."
4,4,"Vai sair até na Globo, algumas pessoas vão cai..."
...,...,...
29849,29849,"faça um bem pra vc mesmo, não assista jornais ..."
29850,29850,eu não aguento a globo repetindo que não tem m...
29851,29851,coloquem na globo ae ta dando reportagem massa...
29852,29852,serio que ta dando na globo? o filme francês a...


In [89]:
rows_train = len(df_train.index)
rows_test = len(df_test.index)
rows_label=rows_train+rows_test

In [90]:
rows_unlabel = len(df_u.index)
rows_unlabel

29854

In [91]:
frames = [df_train.Text, df_test.Text, df_u.Text]
X = pd.concat(frames)
X

0        Quero saber é da lei pros mandantes dos jogos ...
1        Como não amar a carinha de deboche do golpista...
2        Aquela emissora empregada da Globo, que se cha...
3                                 RRRRREDE GLOBO APRESENTA
4        Gente a própria Globo já se pronunciou negando...
                               ...                        
29849    faça um bem pra vc mesmo, não assista jornais ...
29850    eu não aguento a globo repetindo que não tem m...
29851    coloquem na globo ae ta dando reportagem massa...
29852    serio que ta dando na globo? o filme francês a...
29853           tim maia na globo, só isso tenho pra falar
Name: Text, Length: 35867, dtype: object

In [92]:
#vetoriza conjunto completo
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer( )
X_counts = count_vect.fit_transform(X)
X_counts.shape

(35867, 36916)

In [93]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
X_tf = tf_transformer.transform(X_counts)
X_tf.shape
X_ori = X_tf
X_ori.shape

(35867, 36916)

In [94]:
x_train = X_ori[0:rows_train,:]
x_test = X_ori[rows_train:rows_train+rows_test,:]
x_test.shape


(1504, 36916)

In [95]:
#y_ori = df.Polarity
y_train = df_train.Polarity
y_test = df_test.Polarity
y_test.shape

(1504,)

In [96]:
#classificação
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
model = LogisticRegression(random_state=0)
model.fit(x_train,y_train)
pred_values = model.predict(x_test)     
acc = accuracy_score(pred_values , y_test)
acc

0.7865691489361702

In [97]:
f1_score(y_test, pred_values, average='macro')

0.7823665046081858

In [98]:
precision_score(y_test, pred_values, average='macro')

0.7851645543837826

In [99]:
recall_score(y_test, pred_values, average='macro')

0.7807889006675643

In [100]:
#predição dos não-rotulados
X_label = X_ori[rows_label:rows_label+ rows_unlabel]
pred_values_label= model.predict(X_label) 

In [101]:
#avalia a probabilidade de estar certo
prob_label = model.predict_proba(X_label) 


In [102]:
#seleciona os itens que vão entrar no conjunto de treino
threshold = 0.4
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração
rows_train = x_train.shape[0]
rows_train

4509

In [103]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    

In [104]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [105]:
y= np.asarray(y_train)
index_list=[x for x in range(0, rows_train)] #indices para o novo X a partir do X original
iteractions = 0
position = 0
index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

while iteractions < 0.99:
    iteractions = iteractions + threshold
    for i in range(int(t_size/2)):
        #pega um a favor
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 1) #empilha o y
        index_labels.remove(index+rows_label)
        #pega um contra
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 0) #empilha o y
        index_labels.remove(index+rows_label)
        #anda uma posição nos dois 
        position+=1

    X_train_novo = X_ori[index_list,:]#atualiza o X rotulado com os índices que estão em index_list
    X_label = X_ori[index_labels,:] #pega só os índices que sobraram
    rows_unlabel = X_label.get_shape()[0] #novo número de linhas sem rótulo
    rows_label = X_train_novo.get_shape()[0] + rows_test #linhas com rótulo novas + as de teste que estão guardadas
    X_ori = sparse.vstack([X_train_novo, x_test, X_label]) #empilha treino, teste e unlabelled
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)] #atualiza os unlabelled
    position = 0

    model_novo = LogisticRegression(random_state=0)
    model_novo.fit(X_train_novo,y)
    #testa nos conjuntos originais
    pred_values_novo = model_novo.predict(x_test)     
    acc = accuracy_score(pred_values_novo , y_test)
    print("iteraction:",iteractions, "accuracy:", acc)
    
    pred_values_label= model_novo.predict(X_label) 
    prob_label = model.predict_proba(X_label) 
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
        (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)    
    
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

iteraction: 0.4 accuracy: 0.7613031914893617


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.8 accuracy: 0.75


IndexError: list index out of range

33902